# Monochromatic beam reconstruction

In this notebook, the raw time-averaged scan data from APS is converted into projected density and tomographically reconstructed into 2D slices.

---

## Environment setup

In [1]:
import ipywidgets as widgets
import numpy as np

from ipywidgets import (
                        fixed,
                        interact
                        )

from mb_utilities import (
                          convertMB,
                          showSprayMB,
                          scaleTIM,
                          rotCenter,
                          recon,
                          showHistory,
                          showComparisonMB,
                          showLVF
                          )


# Data directories
inp_fld = 'inputs'
out_fld = 'outputs'

### Version information

In [2]:
%reload_ext watermark
%watermark -p ipywidgets,astra,cv2,h5py,numpy,matplotlib,skimage,tomopy

ipywidgets: 7.6.5
astra     : 2.0.0
cv2       : 4.5.1
h5py      : 3.6.0
numpy     : 1.17.0
matplotlib: 3.4.3
skimage   : 0.18.1
tomopy    : 1.7.1



## Projected density scans

The raw scans for both $Re$ cases (open rim condition at $Re$ = 6,700; impact wave condition at $Re$ = 10,100) are located in the `<inputs/*monochromatic*.hdf5>` files. For data compression, the temporally-resolved scan datasets were averaged in time. The reconstructions end up being time-averaged due to the single line of sight nature of the technique, so the loss in resolution is considered unimportant here.

Let's load in the data and convert the scans to projected density using `convertMB()` (see: [mb_utilities.py](./mb_utilities.py)). The datasets are split up by their corresponding axial locations; at the open rim condition $Re$ = 6,700 three locations of $y/d$ = -2, 0, and 2 were collected, where $y$ is the axial location in mm and $d$ is the jet diameter in mm. These are denoted here as the `jets`, `impingement`, and `near` locations. As the `jets` are considered to be stable regardless of flow rate, only the `impingement` and `near` locations were collected at the impact wave condition $Re$ = 10,100 in an effort to save time while running the experiment at APS.

In [3]:
# HDF5 files
path_lo = 'spray_monochromatic_Re-6700_averaged'
path_hi = 'spray_monochromatic_Re-10100_averaged'

# Locations
loc_lo = ['jets', 'impingement', 'near']
loc_hi = ['impingement', 'near']

# Offset indices
ind_lo = [-2, -7, -3]
ind_hi = [-10, -10, -10]

# Retrieve data for the Re = 6,700 case
mb_lo = np.array([
                  convertMB(path_lo, loc, ind)
                  for (loc, ind) in zip(loc_lo, ind_lo)
                  ], dtype=object)

# Retrieve data for the Re = 10,100 case
mb_hi = np.array([
                  convertMB(path_hi, loc, ind)
                  for (loc, ind) in zip(loc_hi, ind_hi)
                  ], dtype=object)

The next step is to scale the liquid masses from each view to be consistent. This is done by calculating the total liquid mass in each view and scaling to the mean liquid mass at each axial location. We see that the variation between all views in liquid mass is around ~2%, however after correction the variation reduces down to effectively 0%.

In [4]:
scaled_lo, err_lo, corr_lo = zip(*[scaleTIM(x) for x in mb_lo])
mb_corr_lo = mb_lo.copy()
mb_corr_lo[:,2] = scaled_lo
print(f'Re = 6700 | Variation in liquid masses: {np.array(err_lo).round(2)}%')
print(f'Re = 6700 | Variation after correction: {np.array(corr_lo).round(2)}%')

scaled_hi, err_hi, corr_hi = zip(*[scaleTIM(x) for x in mb_hi])
mb_corr_hi = mb_hi.copy()
mb_corr_hi[:,2] = scaled_hi
print(f'Re = 10100 | Variation in liquid masses: {np.array(err_hi).round(2)}%')
print(f'Re = 10100 | Variation after correction: {np.array(corr_hi).round(2)}%')

Re = 6700 | Variation in liquid masses: [1.26 1.58 2.56]%
Re = 6700 | Variation after correction: [0. 0. 0.]%
Re = 10100 | Variation in liquid masses: [1.65 1.38]%
Re = 10100 | Variation after correction: [0. 0.]%


Let's visualize the spray at the different axial locations and rotations next. Due to the focused beam nature, each of the points of the spray was collected by raster scanning the spray geometry across the fixed beam. The angular rotations were achieved by rotating the spray geometry using a rotation stage mount.

In [5]:
interact(
         showSprayMB,
         loc=widgets.ToggleButtons(options=list(zip(loc_lo, [0,1,2])),
                                   description='y location',
                                   ),
         view=widgets.IntSlider(min=0, max=17, step=1),
         mb=fixed(mb_corr_lo),
         Re=fixed(6700),
         )

interactive(children=(ToggleButtons(description='y location', options=(('jets', 0), ('impingement', 1), ('near…

<function mb_utilities.showSprayMB(loc, mb, view, Re)>

In [6]:
interact(
         showSprayMB,
         loc=widgets.ToggleButtons(options=list(zip(loc_hi, [0,1])),
                                   description='y location',
                                   ),
         view=widgets.IntSlider(min=0, max=16, step=1),
         mb=fixed(mb_corr_hi),
         Re=fixed(10100),
         )

interactive(children=(ToggleButtons(description='y location', options=(('impingement', 0), ('near', 1)), value…

<function mb_utilities.showSprayMB(loc, mb, view, Re)>

## Time-averaged reconstruction

As can be seen in the scans above, the rotation center of the scans are not all consistent. This is most likely due to the rotation stage center not being perfectly in-line with the focused beam. However, this offset in rotation center can be accounted for, which is done below using `rotCenter()` (see: [mb_utilities.py](./mb_utilities.py)). The function is a wrapper to TomoPy's `find_center()`.

In [7]:
%%capture

c0_lo = [17, 25, 24]
c0_hi = [25, 30]

center_lo = [rotCenter(mb, c0) for (mb, c0) in zip(mb_lo, c0_lo)]
center_hi = [rotCenter(mb, c0) for (mb, c0) in zip(mb_hi, c0_hi)]

For reconstruction, we will need to define the grids for each axial location, as done below. These were found through manual iteration.

In [8]:
# Reconstruction windows
win_lo = [
          [(-4,4), (-4,4)],
          [(-4,4), (-6,6)],
          [(-4,4), (-9,9)],
          ]

win_hi = [
          [(-5,5), (-6,6)],
          [(-5,5), (-12,12)]
          ]

The reconstruction routine is done using a parallel beam assumption with the ASTRA toolbox [[1]](#References), [[2]](#References). An iterative maximum-likelihood expectation-maximization (MLEM) routine is used with an initial volume calculated using a multiplicative line of sight (MLOS) approach [[3]](#References), [[4]](#References), [[5]](#References), [[6]](#References). Smoothing is done between iterations using a Savitzky-Golay 2D filter [[7]](#References). The reconstruction is carried out in `recon()` (see: [mb_utilities.py](./mb_utilities.py)).

In [9]:
v_lo, reproj_lo, hist_lo = zip(*[
                                 recon(mb, win, center)
                                 for (mb, win, center) in zip(mb_lo, win_lo, center_lo)
                                 ])

v_hi, reproj_hi, hist_hi = zip(*[
                                 recon(mb, win, center)
                                 for (mb, win, center) in zip(mb_hi, win_hi, center_hi)
                                 ])

We can inspect the solution history per iteration for each of the reconstructions. The errors between the projection and reprojections are tabulated as the NRMSE (normalized root-mean-squared error in intensity), liquid mass, PSNR (peak signal-to-noise ratio), and SSIM (structural similarity index measure). The convergence is tabulated as the change in the $L_2$-norm in the projected views between iterations. The solution is run until all views reach a change in the $L_2$-norm of less than the equivalent of ~0.5 μg or until 1000 iterations is reached, whichever comes first.

In [10]:
interact(
         showHistory,
         loc=widgets.ToggleButtons(options=list(zip(loc_lo, [0,1,2])),
                                   description='y location',
                                   ),
         hist=fixed(hist_lo),
         Re=fixed(6700),
         )

interactive(children=(ToggleButtons(description='y location', options=(('jets', 0), ('impingement', 1), ('near…

<function mb_utilities.showHistory(hist, loc, Re)>

In [11]:
interact(
         showHistory,
         loc=widgets.ToggleButtons(options=list(zip(loc_hi, [0,1])),
                                   description='y location',
                                   ),
         hist=fixed(hist_lo),
         Re=fixed(10100),
         )

interactive(children=(ToggleButtons(description='y location', options=(('impingement', 0), ('near', 1)), value…

<function mb_utilities.showHistory(hist, loc, Re)>

## Visualization

We can visualize our results by overlaying the reconstructed slice's reprojection scans over the measured projections. This is done through the `showComparisonMB()` function below (see: [mb_utilities.py](./mb_utilities.py)). We see that the reprojections generally follows the shape of the measured projection scans, however they are smoother in shape which is to be expected due to the per-iteration Savitzky-Golay smoothing used to aid in convergence.

In [12]:
interact(
         showComparisonMB,
         loc=widgets.ToggleButtons(options=list(zip(loc_lo, [0,1,2])),
                                   description='y location',
                                   ),
         view=widgets.IntSlider(min=0, max=17, step=1),
         mb=fixed(mb_corr_lo),
         reproj=fixed(reproj_lo),
         Re=fixed(6700),
         )

interactive(children=(ToggleButtons(description='y location', options=(('jets', 0), ('impingement', 1), ('near…

<function mb_utilities.showComparisonMB(mb, reproj, loc, view, Re)>

In [13]:
interact(
         showComparisonMB,
         loc=widgets.ToggleButtons(options=list(zip(loc_hi, [0,1])),
                                   description='y location',
                                   ),
         view=widgets.IntSlider(min=0, max=16, step=1),
         mb=fixed(mb_corr_hi),
         reproj=fixed(reproj_hi),
         Re=fixed(10100),
         )

interactive(children=(ToggleButtons(description='y location', options=(('impingement', 0), ('near', 1)), value…

<function mb_utilities.showComparisonMB(mb, reproj, loc, view, Re)>

The reconstructed slices from `recon()` give slice arrays that have units of reconstructed liquid density ($\rho$ = 997 µg/mm$^3$). These are converted into liquid volume fraction (LVF) by dividing by the bulk liquid density and shown below through `showLVF()` (see: [mb_utilities.py](./mb_utilities.py)). These LVF values give a statistical indication of where the liquid is most likely expected to be distributed$-$due to the time-averaged nature of the measurement, a range of LVF values from 0 (no liquid is expected to be present) up to 1 (liquid is always expected to be present) can exist at these spray conditions. As can be expected, there is a liquid core structure (higher LVF) towards the center of the domain where the impingement point is located.

In [14]:
interact(
         showLVF,
         loc=widgets.ToggleButtons(options=list(zip(loc_lo, [0,1,2])),
                                   description='y location',
                                   ),
         v=fixed(v_lo),
         win=fixed(win_lo),
         Re=fixed(6700),
         )

interactive(children=(ToggleButtons(description='y location', options=(('jets', 0), ('impingement', 1), ('near…

<function mb_utilities.showLVF(v, win, loc, Re)>

In [15]:
interact(
         showLVF,
         loc=widgets.ToggleButtons(options=list(zip(loc_hi, [0,1])),
                                   description='y location',
                                   ),
         v=fixed(v_hi),
         win=fixed(win_hi),
         Re=fixed(10100),
         )

interactive(children=(ToggleButtons(description='y location', options=(('impingement', 0), ('near', 1)), value…

<function mb_utilities.showLVF(v, win, loc, Re)>

## Next steps

This concludes the monochromatic beam reconstruction workflow! Further data analysis on the slice datasets can be done on the NumPy arrays (`v_lo`, `v_hi`), with visualization of the slices and projections through a tool like Matplotlib directly in Python.

## References

1. van Aarle, W. et al. The ASTRA Toolbox: A platform for advanced algorithm development in electron tomography. *Ultramicroscopy* 157, 35–47 (2015). doi: [10.1016/j.ultramic.2015.05.002](https://doi.org/10.1016/j.ultramic.2015.05.002).

2. van Aarle, W. et al. Fast and flexible X-ray tomography using the ASTRA toolbox. *Opt. Express* 24, 25129 (2016). doi: [10.1364/OE.24.025129](https://doi.org/10.1364/OE.24.025129).

3. Jingyu Cui, Pratx, G., Bowen Meng & Levin, C. S. Distributed MLEM: An Iterative Tomographic Image Reconstruction Algorithm for Distributed Memory Architectures. *IEEE Trans. Med. Imaging* 32, 957–967 (2013). doi: [10.1109/TMI.2013.2252913](https://doi.org/10.1109/TMI.2013.2252913).

4. Slomski, A. et al. 3D PET image reconstruction based on the maximum likelihood estimation method (MLEM) algorithm. *Bio-Algorithms and Med-Systems* 10, 1–7 (2014). doi: [10.1515/bams-2013-0106](https://doi.org/10.1515/bams-2013-0106).

5. Atkinson, C. & Soria, J. An efficient simultaneous reconstruction technique for tomographic particle image velocimetry. *Exp. Fluids* 47, 553–568 (2009). doi: [10.1007/s00348-009-0728-0](https://doi.org/10.1007/s00348-009-0728-0).

6. Worth, N. A. & Nickels, T. B. Acceleration of Tomo-PIV by estimating the initial volume intensity distribution. *Exp. Fluids* 45, 847–856 (2008). doi: [10.1007/s00348-008-0504-6](https://doi.org/10.1007/s00348-008-0504-6).

7. Shekhar, C. On simplified application of multidimensional Savitzky-Golay filters and differentiators. in *AIP Conference Proceedings* 020014 (2016). doi: [10.1063/1.4940262](https://doi.org/10.1063/1.4940262).

---

**Author**: Naveed Rahman, Purdue University, 23 March 2022

---